# Scenario 3: Multiple data scientists working on multiple ML models
MLflow setup:

- Tracking server: yes, remote server (EC2).
- Backend store: postgresql database.
- Artifacts store: s3 bucket.
  
The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server.

In [1]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "ahmadluay9" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-52-221-199-80.ap-southeast-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-52-221-199-80.ap-southeast-1.compute.amazonaws.com:5000'


In [3]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifact-remote-ahmadluay9-ap-southeast-1/1', creation_time=1718332635414, experiment_id='1', last_update_time=1718332635414, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifact-remote-ahmadluay9-ap-southeast-1/0', creation_time=1718332348312, experiment_id='0', last_update_time=1718332348312, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 's3://mlflow-artifact-remote-ahmadluay9-ap-southeast-1/1/d7f94b83183e44ba99d234bebddc5af0/artifacts'


In [5]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [9]:
# Retrieve the list of registered models
registered_models = client.search_registered_models()

# Print the names of the registered models
for model in registered_models:
    print(model.name)

iris-classifier


In [8]:
# List runs in the specified experiment
runs = client.search_runs(experiment_ids=['1'])

# Get the run ID of the first run
run_id = runs[0].info.run_id

# Register the model
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/06/14 06:53:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1718348009389, current_stage='None', description='', last_updated_timestamp=1718348009389, name='iris-classifier', run_id='d7f94b83183e44ba99d234bebddc5af0', run_link='', source='s3://mlflow-artifact-remote-ahmadluay9-ap-southeast-1/1/d7f94b83183e44ba99d234bebddc5af0/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>